### the model performing bad because the nature of the training data is different from the test data
the training contains random images from classes to be classified as 0
the validation and test contains random images from other classes so some of them the model has never seen before


## future approach
- [ ] update extract keypoints function to extract fewer keypoints
- [ ] try with 20 frames and 32 frames and 60 frames
- [ ] read more about machine learning to understand why this is falling 
- [ ] read more about one vs many and how to work with data 
- [ ] change loss function to focus more on not picking signs 
- [ ] change optimizer and use lr scheduler
- [ ] use data augemntation
- [ ] for the 0 class get random different data each epoch
- [ ] change binary crossentropy to sparse

# 1 - install dependencies

In [1]:
import os
import sys
import numpy as np
import cv2 
import mediapipe as mp
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt
import time
import pandas as pd

In [2]:
sys.path.append(os.path.abspath(os.path.join('..','..'))) # .. should point to the Utils directory location
from Utils.poseEstimation import mediapipe_detection,draw_styled_landmarks,extract_keypoints

# Move them to utils

In [3]:
DataPath = os.path.join("..","..","data","Datasets")
n_actions = 20


# this function responsible for loading 4 labels files train,validation,test,class_id
# input -> main_dir , output->dictionary containig 4 pandas DataFrames
def load_csv_labels(data_path):
    train_labels = pd.read_csv(os.path.join(data_path,"train_labels.csv"),names=['sample','id'])
    validation_labels = pd.read_csv(os.path.join(data_path,"validation_labels.csv"),names=['sample','id'])
    test_labels = pd.read_csv(os.path.join(data_path,"test_labels.csv"),names=['sample','id'])
    class_id = pd.read_csv(os.path.join(data_path,"class_id.csv"))
    
    return {"train":train_labels,"val":validation_labels,"test":test_labels},class_id

# check if file exists or not input -> full path , output : boolean
def check_file_exists(file_path):
    try:
        f = open(file_path)
        f.close()
        return True
    except IOError:
        return False

# take main_dir,label(train,val,test),file_name and create full path
def get_full_path(data_path,label,file_path):
    return os.path.join(data_path,label,file_path+"_color.mp4")

# input -> number of actions and DataFrame of class_id and return list of n actions with their labels as dictionary
def get_actions_names(n_actions,class_id):
    n_actions = 20
    actions = list(class_id['EN'][:n_actions])
    dic={}
    for i,action in enumerate(actions):
        dic[action]=i
    
    return actions,dic


# take id and label and load all data from it as array of full pathes and labels (the array of only this id)
def get_data_id(action_id,labels,label='train',desired_label=None):
    if(not desired_label):
        desired_label = action_id
        
    label_dic = labels[label]
    data =  label_dic[label_dic['id']==action_id]
    lis =  [get_full_path(DataPath,label,file_name) for file_name in  (data['sample'])]
    data =  [i for i in lis if check_file_exists(i)]
    return data,[desired_label for i in data]

# same as the previous one except it loads all data except the one with this id
def get_data_nid(action_id,labels,label,max_samples=150,desired_label=0):
    label_dic = labels[label]
    data = label_dic[label_dic['id']!=action_id]
    lis =  [get_full_path(DataPath,label,file_name) for file_name in  (data['sample'])]
    data =  [i for i in lis[:max_samples] if check_file_exists(i)]
    return data,[desired_label for i in data]


def create_trainig_validation_test_data(n_actions,labels):
    names = list(labels.keys())

    X_Data  = []
    Y_Data  = []

    for i,name in enumerate(names):
        temp_x = []
        temp_y = []
        for i in range(n_actions):
            x,y = get_data_id(i,labels,name)
            temp_x.extend(x)
            temp_y.extend(y)
        X_Data.append(temp_x)
        Y_Data.append(temp_x)

        
    return X_Data[0],Y_Data[0],X_Data[1],Y_Data[1],X_Data[2],Y_Data[2]


def create_trainig_validation_test_data_binary(action_id,labels,desired_label):
    names = list(labels.keys())

    X_Data  = []
    Y_Data  = []

    for i,name in enumerate(names):
        x,y = get_data_id(action_id,labels,name,desired_label=1)
        x2,y2 = get_data_nid(action_id,labels,name,desired_label=0)
        x.extend(x2)
        y.extend(y2)
        
        
        
        X_Data.append(x)
        Y_Data.append(y)

        
    return X_Data[0],Y_Data[0],X_Data[1],Y_Data[1],X_Data[2],Y_Data[2]

    

In [4]:




labels,class_id  = load_csv_labels(DataPath)


actions,dic = get_actions_names(n_actions,class_id)

# train_X,train_Y,val_X,val_Y,test_X,test_Y = create_trainig_validation_test_data(n_actions,labels)
train_X,train_Y,val_X,val_Y,test_X,test_Y = create_trainig_validation_test_data_binary(0,labels,desired_label=1)

    




In [5]:
print(len(train_X),len(train_Y),len(val_X),len(val_Y),len(test_X),len(test_Y))

276 276 170 170 166 166


### 3.2 process videos into frames and label to cateogries

In [6]:
def capture_frames(video_path):
    video = cv2.VideoCapture(video_path)
    video_length = int(video.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    # count = 0
    # frame_rate = video.get(cv2.CAP_PROP_FRAME_COUNT)

    count=0
    frames = []
    while video.isOpened():
      # frame_id = video.get(0)
        ret, frame = video.read()
        
        if not ret:
            continue
        frames.append(frame)
        count += 1
        # If there are no more frames left
        if (count > (video_length-1)):
          # Release the feed
          video.release()
        # if frame_id % math.floor(1) == 0:
        #     file_name = f"{video_path}_frame_{count}.jpg"
        #     count += 1
        #     # print(f"File name: {file_name}")
        #     frames.append(frame)
        #     # cv2.imwrite(file_name, frame)
    video.release()
    return np.array(frames)


def get_frames(video_path,num_frames):
    frames = capture_frames(video_path)
    video_length = len(frames)
    steps = video_length/num_frames
    count=0
    new_frames=[]
    while count<video_length:
        frame = frames[int(count)]
        new_frames.append(frame)
        count+=steps

    s = np.array(new_frames[:num_frames])
    return np.array(new_frames[:num_frames])


def extract_keypoints_video(path,i=-1):
    
    if(i):
        print(i+" "*20,end="\r")
    frames = get_frames(path,25)
    output_key_points=[]
    with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame in frames:
            image, results = mediapipe_detection(frame, holistic)
            output_key_points.append(extract_keypoints(results))
    return np.array(output_key_points)


def extract_and_save(data,file_path):
    if file_path in os.listdir():
        return np.load(file_path)
    else:
        x =  np.array([extract_keypoints_video(path,f'Saving in {file_path} iteration : {i+1}/{len(data)}') for i,path in enumerate(data) ])
        np.save(file_path,x)
        return x

def save_if_not_exists(data,file_path):
    if file_path in os.listdir():
        return np.load(file_path)
    else:
        np.save(file_path,data)
        return np.array(data)

    

### 3.3 - extract the keypoints and save them as npy or load them if they are exists

In [7]:
train_X = extract_and_save(train_X,"train_X.npy")
val_X   = extract_and_save(val_X,"val_X.npy")
test_X  = extract_and_save(test_X,"test_X.npy")

train_Y = save_if_not_exists(np.array(train_Y),"train_Y.npy")
val_Y   = save_if_not_exists(np.array(val_Y),"val_Y.npy")
test_Y  = save_if_not_exists(np.array(test_Y),"test_Y.npy")


In [8]:
print(len(train_X),len(train_Y),len(val_X),len(val_Y),len(test_X),len(test_Y))

276 276 170 170 166 166


In [9]:
x = np.random.permutation(len(train_X))
train_X = train_X[x]
train_Y = train_Y[x]
x = np.random.permutation(len(val_X))
val_X = val_X[x]
val_Y = val_Y[x]
x = np.random.permutation(len(test_X))
test_X = test_X[x]
test_Y = test_Y[x]

In [10]:
print(len(train_X),len(train_Y),len(val_X),len(val_Y),len(test_X),len(test_Y))

276 276 170 170 166 166


# 4 - build the model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)



In [13]:
input_layer = Input(shape=(25,258))
layer = LSTM(32,return_sequences=True,activation="relu")(input_layer)
layer = LSTM(64,return_sequences=False,activation="relu")(layer)
layer = Dense(64,activation="relu")(layer)
layer = Dense(32,activation="relu")(layer)
layer = Dense(1,activation="sigmoid")(layer)

model = Model(inputs=input_layer,outputs=layer)

In [14]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 25, 258)]         0         
                                                                 
 lstm_2 (LSTM)               (None, 25, 32)            37248     
                                                                 
 lstm_3 (LSTM)               (None, 64)                24832     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 68,353
Trainable params: 68,353
Non-trainable

In [17]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

In [18]:
model.fit(train_X, train_Y, epochs=200, callbacks=[tb_callback],validation_data=(val_X,val_Y))

Epoch 1/200
9/9 [==============================] - 2s 67ms/step - loss: 0.6286 - binary_accuracy: 0.7065 - val_loss: 0.6369 - val_binary_accuracy: 0.6235
Epoch 2/200
9/9 [==============================] - 0s 42ms/step - loss: 0.5317 - binary_accuracy: 0.7246 - val_loss: 0.4930 - val_binary_accuracy: 0.9059
Epoch 3/200
9/9 [==============================] - 0s 41ms/step - loss: 0.4245 - binary_accuracy: 0.8623 - val_loss: 1.1674 - val_binary_accuracy: 0.4176
Epoch 4/200
9/9 [==============================] - 0s 41ms/step - loss: 0.3696 - binary_accuracy: 0.8551 - val_loss: 1.5727 - val_binary_accuracy: 0.1059
Epoch 5/200
9/9 [==============================] - 0s 41ms/step - loss: 0.2958 - binary_accuracy: 0.8949 - val_loss: 1.4922 - val_binary_accuracy: 0.3706
Epoch 6/200
9/9 [==============================] - 0s 41ms/step - loss: 0.2287 - binary_accuracy: 0.9203 - val_loss: 0.6217 - val_binary_accuracy: 0.7118
Epoch 7/200
9/9 [==============================] - 0s 43ms/step - loss: 0.74

KeyboardInterrupt: 

In [138]:
model.save_weights("model2.h5")

In [139]:
model.load_weights('model2.h5')

In [140]:
train_X.shape = (-1,6450)
val_X.shape = (-1,6450)
test_X.shape = (-1,6450)

In [141]:

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(train_X, train_Y)

print(clf.score(train_X, train_Y))
print(clf.score(val_X, val_Y))
print(clf.score(test_X, test_Y))


1.0
0.23529411764705882
0.13253012048192772


/home/mina/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# 5 - Make predictions

In [142]:
train_X.shape = (-1,25,258)
val_X.shape = (-1,25,258)
test_X.shape = (-1,25,258)

### this doesn't make any sence

In [143]:
model.evaluate(test_X,test_Y)

6/6 [==============================] - 0s 7ms/step - loss: 8.4343 - binary_accuracy: 0.1024


[8.434293746948242, 0.10240963846445084]

# confusion matrix

In [144]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score,plot_confusion_matrix

yhat = model.predict(val_X)
# ytrue = np.argmax(y_test, axis=1).tolist()
#yhat = np.argmax(yhat, axis=1).tolist()

multilabel_confusion_matrix(val_Y, yhat.round())

array([[[ 20,   0],
        [139,  11]],

       [[ 11, 139],
        [  0,  20]]])

In [68]:
for i in range(len(yhat)):
    print(yhat[i],val_Y[i]) if yhat[i].round() != val_Y[i] else None

[0.99988556] 0
[0.99930036] 0
[0.9642273] 0
[0.9993154] 0
[0.9998085] 0
[0.7735257] 0
[0.99989057] 0
[0.9998754] 0
[0.99961984] 0
[0.7913579] 0
[0.9974141] 0
[0.99890935] 0
[0.952731] 0
[0.99667335] 0
[0.9997353] 0
[0.99817073] 0
[0.99983644] 0
[0.9997334] 0
[0.99978095] 0
[0.9912566] 0
[0.9998524] 0
[0.84857273] 0
[0.9997689] 0
[0.9976794] 0
[0.98566055] 0
[0.9999031] 0
[0.9994234] 0
[0.99890506] 0
[0.99978834] 0
[0.9997713] 0
[0.99995995] 0
[0.9977958] 0
[0.9982834] 0
[0.99965656] 0
[0.99978286] 0
[0.9991611] 0
[0.9998036] 0
[0.9992513] 0
[0.99787617] 0
[0.9996979] 0
[0.9115199] 0
[0.99982834] 0
[0.99982494] 0
[0.99979144] 0
[0.99243647] 0
[0.9959235] 0
[0.86173433] 0
[0.9986154] 0
[0.99976116] 0
[0.98479164] 0
[0.99989605] 0
[0.99900705] 0
[0.99982786] 0
[0.9997954] 0
[0.99935585] 0
[0.9871301] 0
[0.9992724] 0
[0.9993381] 0
[0.99968004] 0
[0.99991953] 0
[0.99917954] 0
[0.9996544] 0
[0.9998778] 0
[0.9959329] 0
[0.94264627] 0
[0.99973756] 0
[0.9998772] 0
[0.74445236] 0
[0.99324083] 0


# 6 - test in real time

In [145]:
from scipy import stats

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [147]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        if(not ret):
            break
    
        frame = cv2.resize(frame,(512,512))
    
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-25:]
        
        if len(sequence) == 25:
            x = np.expand_dims(sequence, axis=0)
#             x.shape = (1,25*258)
#             res = clf.predict(x)
#             print(res)
            # print(x.shape)
            res = model.predict(x)[0][0] # res is now number between 0 and 1 where 1 mean sign
    
            predictions.append(res.round())
            

            
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, str(res), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [112]:
cap.release()

In [113]:
cv2.destroyAllWindows()